In [ ]:
#%pip install stable-baselines3==1.2.0

In [ ]:
#%pip install gym==0.25.2

In [ ]:
#%pip install gym-super-mario-bros==7.3.0

In [ ]:
#%pip install opencv-python

In [7]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT,RIGHT_ONLY
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [ ]:
CHECKPOINT_DIR = './train'
LOG_DIR = './logs'

model = PPO.load('./train/best_model_10000')

In [3]:
def allWorldStagesTesting():
    worlds = [1, 2, 3, 4, 5, 6, 7, 8]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [4]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

env_names = [name for name in allWorldStagesTesting()]
print(env_names)

def allWorldStages():
    worlds = [1, 2, 3, 4, 5, 6, 7, 8]
    #Training on Stages 1-3 for each world, testing on final stage of each world
    stages = [1, 2, 3]

    for world in worlds:
        for stage in stages:
            environment_name = f"SuperMarioBros-{world}-{stage}-v0"
            yield environment_name

env_names = [name for name in allWorldStages()]
print(env_names)

def make_env(env_name):
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    env = GrayScaleObservation(env, keep_dim=True)
    return env

['SuperMarioBros-1-4-v0', 'SuperMarioBros-2-4-v0', 'SuperMarioBros-3-4-v0', 'SuperMarioBros-4-4-v0', 'SuperMarioBros-5-4-v0', 'SuperMarioBros-6-4-v0', 'SuperMarioBros-7-4-v0', 'SuperMarioBros-8-4-v0']
['SuperMarioBros-1-1-v0', 'SuperMarioBros-1-2-v0', 'SuperMarioBros-1-3-v0', 'SuperMarioBros-2-1-v0', 'SuperMarioBros-2-2-v0', 'SuperMarioBros-2-3-v0', 'SuperMarioBros-3-1-v0', 'SuperMarioBros-3-2-v0', 'SuperMarioBros-3-3-v0', 'SuperMarioBros-4-1-v0', 'SuperMarioBros-4-2-v0', 'SuperMarioBros-4-3-v0', 'SuperMarioBros-5-1-v0', 'SuperMarioBros-5-2-v0', 'SuperMarioBros-5-3-v0', 'SuperMarioBros-6-1-v0', 'SuperMarioBros-6-2-v0', 'SuperMarioBros-6-3-v0', 'SuperMarioBros-7-1-v0', 'SuperMarioBros-7-2-v0', 'SuperMarioBros-7-3-v0', 'SuperMarioBros-8-1-v0', 'SuperMarioBros-8-2-v0', 'SuperMarioBros-8-3-v0']


In [ ]:
env_names = [name for name in allWorldStagesTesting()]
env_list = [make_env(name) for name in env_names]

env = DummyVecEnv([lambda env_name=name: make_env(env_name) for name in env_names])

env = VecFrameStack(env, 4, channels_order='last')
state = env.reset()
print(state.shape)

In [14]:
state = env.reset()

In [ ]:
while True:
    action, _state, = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()